### Autoencoders and Neural Network for Place recognition with WiFi fingerprints
Implementation of algorithm discussed in <a href="https://arxiv.org/pdf/1611.02049v1.pdf">Low-effort place recognition with WiFi fingerprints using Deep Learning </a>

In [ ]:
import pandas as pd
import numpy as np
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
from sklearn.preprocessing import scale

# get dataset from UCI repository
import requests
import io
from zipfile import ZipFile
response = requests.get('https://archive.ics.uci.edu/ml/machine-learning-databases/00310/UJIndoorLoc.zip')
compressedFile = io.BytesIO(response.content)
zipFile = ZipFile(compressedFile)

In [ ]:
dataset = pd.read_csv(zipFile.open('UJIndoorLoc/trainingData.csv'), header=0)

features = np.asarray(dataset.iloc[:,0:520])
# replace NaN with -110 dBm
features[features == 100] = -110
# feature normalization
features = (features - features.mean()) / features.var()

labels = np.asarray(dataset["BUILDINGID"].map(str) + dataset["FLOOR"].map(str))
# Convert categorical variable into dummy/indicator variables
labels = np.asarray(pd.get_dummies(labels))

#### Dividing UJIndoorLoc training data set into training and validation set

In [ ]:
# create a boolean vector that randomly selects the 70% of data
train_val_split = np.random.rand(len(features)) < 0.70
# training set
train_x = features[train_val_split]
train_y = labels[train_val_split]
# validation set
val_x = features[~train_val_split]
val_y = labels[~train_val_split]

In [ ]:
print("train_x.shape:",train_x.shape)
print("val_x.shape:",val_x.shape)

train_x.shape: (13747, 520)
val_x.shape: (6190, 520)


#### Using UJIndoorLoc validation data set as testing set

In [ ]:
test_dataset = pd.read_csv(zipFile.open('UJIndoorLoc/validationData.csv'), header=0)

test_features = np.asarray(test_dataset.iloc[:,0:520])
test_features[test_features == 100] = -110
test_features = (test_features - test_features.mean()) / test_features.var()

test_labels = np.asarray(test_dataset["BUILDINGID"].map(str) + test_dataset["FLOOR"].map(str))
test_labels = np.asarray(pd.get_dummies(test_labels))

#### Model architecture
Image take from: https://arxiv.org/pdf/1611.02049v1.pdf

![alt text](https://media.springernature.com/lw785/springer-static/image/chp%3A10.1007%2F978-3-319-54042-9_57/MediaObjects/437022_1_En_57_Fig1_HTML.gif)   
![alt text](https://media.springernature.com/lw785/springer-static/image/chp%3A10.1007%2F978-3-319-54042-9_57/MediaObjects/437022_1_En_57_Fig2_HTML.gif)

In [ ]:
def weight_variable(shape):
    initial = tf.random.truncated_normal(shape, stddev = 0.1)
    return tf.Variable(initial)

def bias_variable(shape):
    initial = tf.constant(0.0, shape = shape)
    return tf.Variable(initial)

In [ ]:
n_input = 520 
print("n_input:",n_input)
n_hidden_1 = 256 
print("n_hidden_1:",n_hidden_1)
n_hidden_2 = 128 
print("n_hidden_2:",n_hidden_2)
n_hidden_3 = 64 
print("n_hidden_3:",n_hidden_3)

n_classes = labels.shape[1]
print("n_classes:",n_classes)

learning_rate = 0.00001 
print("learning_rate:",learning_rate)
training_epochs = 30 
print("training_epochs:",training_epochs)
batch_size = 15 
print("batch_size:",batch_size)

total_batches = train_x.shape[0] // batch_size
print("total_batches = train_x.shape[0] // batch_size: ", train_x.shape[0], '/', batch_size, '=', total_batches)

n_input: 520
n_hidden_1: 256
n_hidden_2: 128
n_hidden_3: 64
n_classes: 13
learning_rate: 1e-05
training_epochs: 30
batch_size: 15
total_batches = train_x.shape[0] // batch_size:  13747 / 15 = 916


In [ ]:
X = tf.placeholder(tf.float32, shape=[None,n_input])
Y = tf.placeholder(tf.float32,[None,n_classes])

# --------------------- Encoder Variables --------------- #

e_weights_h1 = weight_variable([n_input, n_hidden_1])
e_biases_h1 = bias_variable([n_hidden_1])

e_weights_h2 = weight_variable([n_hidden_1, n_hidden_2])
e_biases_h2 = bias_variable([n_hidden_2])

e_weights_h3 = weight_variable([n_hidden_2, n_hidden_3])
e_biases_h3 = bias_variable([n_hidden_3])

# --------------------- Decoder Variables --------------- #

d_weights_h1 = weight_variable([n_hidden_3, n_hidden_2])
d_biases_h1 = bias_variable([n_hidden_2])

d_weights_h2 = weight_variable([n_hidden_2, n_hidden_1])
d_biases_h2 = bias_variable([n_hidden_1])

d_weights_h3 = weight_variable([n_hidden_1, n_input])
d_biases_h3 = bias_variable([n_input])

# --------------------- DNN Variables ------------------ #

dnn_weights_h1 = weight_variable([n_hidden_3, n_hidden_2])
dnn_biases_h1 = bias_variable([n_hidden_2])

dnn_weights_h2 = weight_variable([n_hidden_2, n_hidden_2])
dnn_biases_h2 = bias_variable([n_hidden_2])

dnn_weights_out = weight_variable([n_hidden_2, n_classes])
dnn_biases_out = bias_variable([n_classes])

In [ ]:
def encode(x):
    l1 = tf.nn.tanh(tf.add(tf.matmul(x,e_weights_h1),e_biases_h1))
    l2 = tf.nn.tanh(tf.add(tf.matmul(l1,e_weights_h2),e_biases_h2))
    l3 = tf.nn.tanh(tf.add(tf.matmul(l2,e_weights_h3),e_biases_h3))
    return l3
    
def decode(x):
    l1 = tf.nn.tanh(tf.add(tf.matmul(x,d_weights_h1),d_biases_h1))
    l2 = tf.nn.tanh(tf.add(tf.matmul(l1,d_weights_h2),d_biases_h2))
    l3 = tf.nn.tanh(tf.add(tf.matmul(l2,d_weights_h3),d_biases_h3))
    return l3

def dnn(x):
    l1 = tf.nn.tanh(tf.add(tf.matmul(x,dnn_weights_h1),dnn_biases_h1))
    l2 = tf.nn.tanh(tf.add(tf.matmul(l1,dnn_weights_h2),dnn_biases_h2))
    out = tf.nn.softmax(tf.add(tf.matmul(l2,dnn_weights_out),dnn_biases_out))
    return out

In [ ]:
encoded = encode(X)
decoded = decode(encoded) 
y_ = dnn(encoded)

In [ ]:
# unsupervised cost function
us_cost_function = tf.reduce_mean(tf.pow(X - decoded, 2))
# supervised cost function
s_cost_function = -tf.reduce_sum(Y * tf.log(y_))

us_optimizer = tf.train.AdamOptimizer(learning_rate).minimize(us_cost_function)
s_optimizer = tf.train.AdamOptimizer(learning_rate).minimize(s_cost_function)

In [ ]:
correct_prediction = tf.equal(tf.argmax(y_,1), tf.argmax(Y,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [ ]:
with tf.Session() as session:
    tf.global_variables_initializer().run()
    
    # ------------ 1. Training Autoencoders - Unsupervised Learning ----------- #
    for epoch in range(training_epochs):
        epoch_costs = np.empty(0)
        for b in range(total_batches):
            offset = (b * batch_size) % (train_x.shape[0] - batch_size)
            batch_x = train_x[offset:(offset + batch_size), :]
            _, c = session.run([us_optimizer, us_cost_function],feed_dict={X: batch_x})
            epoch_costs = np.append(epoch_costs,c)
        print("Epoch: ",epoch," Loss: ",np.mean(epoch_costs))
    print("Unsupervised pre-training finished...")
    
    
    # ---------------- 2. Training NN - Supervised Learning ------------------ #
    for epoch in range(training_epochs):
        epoch_costs = np.empty(0)
        for b in range(total_batches):
            offset = (b * batch_size) % (train_x.shape[0] - batch_size)
            batch_x = train_x[offset:(offset + batch_size), :]
            batch_y = train_y[offset:(offset + batch_size), :]
            _, c = session.run([s_optimizer, s_cost_function],feed_dict={X: batch_x, Y : batch_y})
            epoch_costs = np.append(epoch_costs,c)
        print("Epoch: ",epoch," Loss: ",np.mean(epoch_costs)," Training Accuracy: ", \
            session.run(accuracy, feed_dict={X: train_x, Y: train_y}), \
            "Validation Accuracy:", session.run(accuracy, feed_dict={X: val_x, Y: val_y}))
            
    print("Supervised training finished...")
    

    print("\nTesting Accuracy:", session.run(accuracy, feed_dict={X: test_features, Y: test_labels}))

Epoch:  0  Loss:  0.06369901477871666
Epoch:  1  Loss:  0.04265240476442314
Epoch:  2  Loss:  0.03308895354842417
Epoch:  3  Loss:  0.027057796860557034
Epoch:  4  Loss:  0.023014783347838897
Epoch:  5  Loss:  0.020166551119314435
Epoch:  6  Loss:  0.01792633680750791
Epoch:  7  Loss:  0.016123377665161566
Epoch:  8  Loss:  0.014718779304022107
Epoch:  9  Loss:  0.013639677218598717
Epoch:  10  Loss:  0.012780552171780748
Epoch:  11  Loss:  0.012068499987904291
Epoch:  12  Loss:  0.011461301624630557
Epoch:  13  Loss:  0.01093232657736997
Epoch:  14  Loss:  0.010465052162091691
Epoch:  15  Loss:  0.010049167842305301
Epoch:  16  Loss:  0.009677799295290378
Epoch:  17  Loss:  0.009346183659874847
Epoch:  18  Loss:  0.009050630925448945
Epoch:  19  Loss:  0.00878755595101914
Epoch:  20  Loss:  0.008553039642227773
Epoch:  21  Loss:  0.00834296344943125
Epoch:  22  Loss:  0.00815338659553695
Epoch:  23  Loss:  0.007980857883258743
Epoch:  24  Loss:  0.00782255021556317
Epoch:  25  Loss:  

--------------------------------------------------------------------------------------------------------------------------